# Only for inference

In [1]:
import h5py
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import _pickle
y_vocab = _pickle.load(open('./data/y_vocab.py3.cPickle', 'rb'))
y_vocab['43>109>1576>-1']
print(len(y_vocab))

4215


In [3]:
token_to_cate = {}
for it in y_vocab.items():
     token_to_cate[it[1]] = it[0]

## import utils

In [4]:
from myUtils.myUtils import *

In [5]:
helper = Helper()
seq_len=100
hash_size=4000

In [6]:
%%time
#  처음만 파일 만들기
#  no... 여기서 helper.df 만드네 코드 바꾸자 ㅜㅜ
for i in range(1,2):
    helper.chunk_no = i
    helper.path = './data_org/dev.chunk.0'+str(helper.chunk_no)
    print('------------%s-----------'%(helper.path))    
    helper.makeDF(0, 1000000, mode="dev")    
    
    tmp_path_x = 'data/infer_x.csv'
#     tmp_path_y = 'data/infer_y.csv'
    data_x = helper.df2data(tmp_path_x, seq_len=seq_len, hash_size=hash_size)
#     data_y = helper.cate2token(tmp_path_y , y_vocab)

------------./data_org/dev.chunk.01-----------


A Jupyter Widget


Wall time: 1min 7s


In [7]:
"""# 다음부턴
df_tmp = pd.read_csv('data/infer_x.csv', header=None)
data_x = df_tmp.as_matrix()"""

"# 다음부턴\ndf_tmp = pd.read_csv('data/infer_x.csv', header=None)\ndata_x = df_tmp.as_matrix()"

## make model

In [8]:
# make DAG
tf.reset_default_graph()
tf.set_random_seed(2018)

# train Parameters
# seq_len = 35
output_dim = 4215 # 1번 트레이닝셋만

epoch = 80
batch_size = 1024
# vocabulary_size = len(vocab)
vocabulary_size = hash_size
embedding_size = 300
# embedding_size = 256
features = 300


# =============================================================================

Y = tf.placeholder(tf.int16, [None, output_dim], name="label")
lr = tf.placeholder(tf.float32, [], name='learning_rate')
keep_prob = tf.placeholder(tf.float32, [], name="keep_prob")

#  embedding
X = tf.placeholder(tf.int32, [None, seq_len], name="word_tokens") 
word_embeddings = tf.get_variable("word_embeddings",
    [vocabulary_size, embedding_size], initializer=tf.contrib.layers.xavier_initializer())
embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, X) # batch * seq * embeddding

"""
#  dropout layer
def _sequence_dropout(step_inputs, keep_prob):
        # apply dropout to each input
        # input : a list of input tensor which shape is [None, input_dim]
        with tf.name_scope('sequence_dropout') as scope:
            step_outputs = []
            for t, _input in enumerate(step_inputs):
                step_outputs.append( tf.nn.dropout(_input, keep_prob) )
        return step_outputs

embedded_word_ids = tf.unstack(embedded_word_ids, axis=1)
step_inputs = _sequence_dropout(embedded_word_ids, keep_prob) # seq * batch * embedding

#  FCN layer
doc_mean = tf.reduce_mean(step_inputs, axis=0) # batch * embedding (mean)
# hint = tf.placeholder(tf.float32, [None, 609], name='hint') # previous category
# bf_lenear = tf.concat([doc_mean, hint], axis=1) # batch * (embediing + hint)
# bf_lenear = tf.nn.dropout(bf_lenear, keep_prob)
"""

embedded_word_ids = tf.reshape(embedded_word_ids, [-1, seq_len, embedding_size, 1])
# CNN layer
# kernel size 3
F1 = tf.get_variable('3gram-filter', [3, embedding_size, 1, features], 
                     initializer=tf.contrib.layers.xavier_initializer()) # (row, column, channel, kernels)
L1 = tf.nn.conv2d(embedded_word_ids, F1, strides=[1, 1, 1, 1], padding='VALID')
L1 = tf.nn.tanh(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, seq_len-2, 1, 1],
                    strides=[1, 1, 1, 1], padding='VALID')

# kernel size 4
F2 = tf.get_variable('4gram-filter', [4, embedding_size, 1, features], 
                     initializer=tf.contrib.layers.xavier_initializer()) # (row, column, channel, kernels)
L2 = tf.nn.conv2d(embedded_word_ids, F2, strides=[1, 1, 1, 1], padding='VALID')
L2 = tf.nn.tanh(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, seq_len-3, 1, 1],
                    strides=[1, 1, 1, 1], padding='VALID')

# kernel size 5
F3 = tf.get_variable('5gram-filter', [5, embedding_size, 1, features], 
                     initializer=tf.contrib.layers.xavier_initializer()) # (row, column, channel, kernels)
L3 = tf.nn.conv2d(embedded_word_ids, F3, strides=[1, 1, 1, 1], padding='VALID')
L3 = tf.nn.tanh(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, seq_len-4, 1, 1],
                    strides=[1, 1, 1, 1], padding='VALID')

L_flat = tf.concat([L1, L2, L3], 3) # (?, 1, 1, features*3)
L_flat = tf.reshape(L_flat, [-1, 3 * features]) # (?, features*3)

# 오캄의 면도날... 굳이 없어도 되는 레이어인듯
# L_linear = tf.contrib.layers.fully_connected(L_flat, 512, activation_fn=tf.nn.relu)
# L_linear = tf.nn.dropout(L_linear, keep_prob)
L_linear = tf.nn.dropout(L_flat, keep_prob)


# Y_pred = tf.contrib.layers.fully_connected(bf_lenear, output_dim, activation_fn=tf.nn.relu)  # We use the last cell's output
Y_pred = tf.contrib.layers.fully_connected(L_linear, output_dim, activation_fn=None)  # We use the last cell's output

# # image feature
# X = tf.placeholder(tf.float32, [None, len(data_x[0])], name="img_feat")
# X = tf.nn.dropout(X, keep_prob)
# Y_pred = tf.contrib.layers.fully_connected(X, output_dim,
#                                            activation_fn=tf.nn.relu, weights_initializer=tf.contrib.layers.xavier_initializer())  # We use the last cell's output

# optimize
cost =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=Y_pred, labels=Y, name='cross_entropy'))
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(cost)

# prediction
predicted = tf.argmax(Y_pred, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, tf.argmax(Y, 1)), dtype=tf.float32))

# saver
name_to_var_map = {var.op.name: var for var in tf.global_variables()}
saver = tf.train.Saver(name_to_var_map, name='my_saver')

In [9]:
# restore model
if 'sess' in globals(): sess.close()
sess = tf.InteractiveSession()

ckpt_path = './small_model/small'
saver.restore(sess, ckpt_path+'-16662')

INFO:tensorflow:Restoring parameters from ./small_model/small-16662


## Make .tsv file

In [10]:
from math import ceil

chunk_size = 10000
iter_num = ceil(len(data_x)/chunk_size)
cate_predicted = []

for cnt in range(iter_num):
    dev = sess.run([predicted], feed_dict={X:data_x[chunk_size*cnt:chunk_size*(cnt+1)], keep_prob:1})
    for i in dev[0]:
        tmp = token_to_cate[i]
        cate_predicted.append(tmp.replace('>', '\t'))
    

assert len(cate_predicted) == len(helper.df['pid'])
with open("cnn-char.predict.tsv", "w") as f:
    for el in zip(helper.df['pid'], cate_predicted):
#         print(el)
        tmp = '\t'.join(el)      
        f.write(tmp+'\n')